# Analiza Wielowymiarowa - Testy parametryczne i nieparametryczne - zajecia 2

In [1]:
from multidim.utils import resolve_stata

stata_conf = resolve_stata(version = 17, stype = "se")
STATA_PATH = stata_conf.path
STATA_TYPE = stata_conf.type

In [2]:
# STATA
import stata_setup
stata_setup.config(STATA_PATH, STATA_TYPE)
from pystata import stata


  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       SE—Standard Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Unlimited-user network, expiring  1 Nov 2022
Serial number: 401709308728
  Licensed to: Maciej Nasinski
               University of Warsaw

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. Maximum number of variables is set to 5,000; see help set_maxvar.


In [3]:
import pandas as pd
import scipy
import numpy as np
import scipy.stats as stats
import sklearn
from statsmodels.stats.proportion import proportions_ztest

### Uzycie staty w jupyterach

%stata dla wywolan jednoliniowych
%%stata dla wywolan wieloliniowych

oba wywolania posiadaja dodatkowe argumenty ktore omowimy podczas zajec
%%stata -d dane -eret myeret

Obiekt stata z cala gama metod takich jak stata.run

### Podstawowe zbiory danych w STATA

In [4]:
%stata help dta_examples


Example datasets installed with Stata
-------------------------------------------------------------------------------
    The datasets listed here are installed with Stata.  You can also see a
    complete list of the datasets used in the Stata documentation that are
    available via the Internet.  Each manual title is listed as a link that
    will take you to the list of datasets for that manual.
-------------------------------------------------------------------------------

        auto.dta            use | describe
        auto2.dta           use | describe
        autornd.dta         use | describe
        bplong.dta          use | describe
        bpwide.dta          use | describe
        cancer.dta          use | describe
        census.dta          use | describe
        citytemp.dta        use | describe
        citytemp4.dta       use | describe
        educ99gdp.dta       use | describe
        gnp96.dta           use | describe
        lifeexp.dta         use | describe

### Wczytywanie danych danych

In [19]:
%stata sysuse ../data/samochody.dta, clear

Exception in thread Stata:
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.10/3.10.6_2/Frameworks/Python.framework/Versions/3.10/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/Applications/Stata/utilities/pystata/core/stout.py", line 169, in run
    raise SystemError(output)
SystemError: file "../data/samochody.dta" not found
r(601);



In [6]:
from multidim.datasets import load_auto
auto = load_auto()

In [27]:
import pandas
pandas.read_stata("../data/tibetan.dta")

,length,breadth,height,upper,face
0,190.5,152.5,145.0,73.5,136.5
1,172.5,132.0,125.5,63.0,121.0
2,167.0,130.0,125.5,69.5,119.5
3,169.5,150.5,133.5,64.5,128.0
4,175.0,138.5,126.0,77.5,135.5
5,177.5,142.5,142.5,71.5,131.0
6,179.5,142.5,127.5,70.5,134.5
7,179.5,138.0,133.5,73.5,132.5
8,173.5,135.5,130.5,70.0,133.5
9,162.5,139.0,131.0,62.0,126.0


In [7]:
?stata.pdataframe_from_data
# auto = stata.pdataframe_from_data(missingval = np.NaN, valuelabel = True)
# i spowrotem
# stata.pdataframe_to_data(auto)

Signature:
stata.pdataframe_from_data(
    var=None,
    obs=None,
    selectvar=None,
    valuelabel=False,
    missingval=_DefaultMissing(),
)
Docstring:
Export values from the current Stata dataset into a pandas DataFrame.

Parameters
----------
var : int, str, or list-like, optional
    Variables to access. It can be specified as a single variable
    index or name, or an iterable of variable indices or names. 
    If `var` is not specified, all the variables are specified.

obs : int or list-like, optional
    Observations to access. It can be specified as a single 
    observation index or an iterable of observation indices. 
    If `obs` is not specified, all the observations are specified.

selectvar : int or str, optional
    Observations for which `selectvar!=0` will be selected. If `selectvar`
    is an integer, it is interpreted as a variable index. If `selectvar` 
    is a string, it should contain the name of a Stata variable. 
    Specifying `selectvar` as "" has the sam

### Opis zawartosci zbioru danych

In [8]:
%stata describe


Contains data from /Applications/Stata/ado/base/a/auto.dta
 Observations:            74                  1978 automobile data
    Variables:            12                  13 Apr 2020 17:45
                                              (_dta has notes)
-------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
-------------------------------------------------------------------------------
make            str18   %-18s                 Make and model
price           int     %8.0gc                Price
mpg             int     %8.0g                 Mileage (mpg)
rep78           int     %8.0g                 Repair record 1978
headroom        float   %6.1f                 Headroom (in.)
trunk           int     %8.0g                 Trunk space (cu. ft.)
weight          int     %8.0gc                Weight (lbs.)
length          int     %8.0g                 Length (in.

### Ksiazka kodowa zbioru

Polecenie codebook pozwala nam zbadac charakterystyki per zmienna.

In [9]:
%%stata
/* help codebook */
/* codebook */
codebook make


. /* help codebook */
. /* codebook */
. codebook make

-------------------------------------------------------------------------------
make                                                             Make and model
-------------------------------------------------------------------------------

                  Type: String (str18), but longest is str17

         Unique values: 74                        Missing "": 0/74

              Examples: "Cad. Deville"
                        "Dodge Magnum"
                        "Merc. XR-7"
                        "Pont. Catalina"


. 


In [10]:
auto.dtypes

make              object
price              int16
mpg                int16
rep78            float64
headroom         float32
trunk              int16
weight             int16
length             int16
turn               int16
displacement       int16
gear_ratio       float32
foreign         category
dtype: object

### Wyswietlenie danych

Polecenie list bez dodatkowych zmiennych powinno byc wywolywane na mniejszych zbiorach.  
W przypadku wiekszych zbiorow przydatne bedzie uzycie dodatkowych argumentow zawezajacych wyszukanie.
W tym przypadku zawezimy wyszukanie samochodow do tych najciezszych.

In [11]:
%%stata
/* help list */
/* list, table */
list if weight > 4500


. /* help list */
. /* list, table */
. list if weight > 4500

     +----------------------------------------------------------------------+
 26. | make              |  price | mpg | rep78 | headroom | trunk | weight |
     | Linc. Continental | 11,497 |  12 |     3 |      3.5 |    22 |  4,840 |
     |----------------------------------------------------------------------|
     |   length   |   turn   |   displa~t   |   gear_r~o    |    foreign    |
     |      233   |     51   |        400   |       2.47    |   Domestic    |
     +----------------------------------------------------------------------+

     +----------------------------------------------------------------------+
 27. | make              |  price | mpg | rep78 | headroom | trunk | weight |
     | Linc. Mark V      | 13,594 |  12 |     3 |      2.5 |    18 |  4,720 |
     |----------------------------------------------------------------------|
     |   length   |   turn   |   displa~t   |   gear_r~o    |    foreign    |

In [12]:
auto.query("weight > 4500")

,make,price,mpg,rep78,headroom,trunk,weight,length,turn,displacement,gear_ratio,foreign
25,Linc. Continental,11497,12,3.0,3.5,22,4840,233,51,400,2.47,Domestic
26,Linc. Mark V,13594,12,3.0,2.5,18,4720,230,48,400,2.47,Domestic


Szybki przeglad zmiennych

In [13]:
%%stata
/* help inspect */
/* inspect */
inspect weight


. /* help inspect */
. /* inspect */
. inspect weight

weight:  Weight (lbs.)                          Number of observations
----------------------                 ---------------------------------------
                                             Total      Integers   Nonintegers
|  #       #                 Negative            -             -             -
|  #       #                 Zero                -             -             -
|  #   #   #   #             Positive           74            74             -
|  #   #   #   #                       -----------   -----------   -----------
|  #   #   #   #             Total              74            74             -
|  #   #   #   #   #         Missing             -
+----------------------                -----------
1760               4840                         74
  (64 unique values)

. 


In [14]:
%%stata
/* Statystyki opisowe */
summarize


. /* Statystyki opisowe */
. summarize

    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
        make |          0
       price |         74    6165.257    2949.496       3291      15906
         mpg |         74     21.2973    5.785503         12         41
       rep78 |         69    3.405797    .9899323          1          5
    headroom |         74    2.993243    .8459948        1.5          5
-------------+---------------------------------------------------------
       trunk |         74    13.75676    4.277404          5         23
      weight |         74    3019.459    777.1936       1760       4840
      length |         74    187.9324    22.26634        142        233
        turn |         74    39.64865    4.399354         31         51
displacement |         74    197.2973    91.83722         79        425
-------------+-------------------------------------------------------

In [15]:
auto.describe()

,price,mpg,rep78,headroom,trunk,weight,length,turn,displacement,gear_ratio
count,74.000000,74.000000,69.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000
mean,6165.256757,21.297297,3.405797,2.993243,13.756757,3019.459459,187.932432,39.648649,197.297297,3.014865
std,2949.495885,5.785503,0.989932,0.845995,4.277404,777.193567,22.266340,4.399354,91.837219,0.456287
min,3291.000000,12.000000,1.000000,1.500000,5.000000,1760.000000,142.000000,31.000000,79.000000,2.190000
25%,4220.250000,18.000000,3.000000,2.500000,10.250000,2250.000000,170.000000,36.000000,119.000000,2.730000
50%,5006.500000,20.000000,3.000000,3.000000,14.000000,3190.000000,192.500000,40.000000,196.000000,2.955000
75%,6332.250000,24.750000,4.000000,3.500000,16.750000,3600.000000,203.750000,43.000000,245.250000,3.352500
max,15906.000000,41.000000,5.000000,5.000000,23.000000,4840.000000,233.000000,51.000000,425.000000,3.890000


## Testy parametryczne i nieparametryczne dla srednich

In [16]:
# %stata help ttest
# ?stats.ttest_ind

test t dla srednich przy zalozeniu rownej wariancji
Warto zwrocic uwage ze roznica wynosi -312.2587 jendnak p-value dla wszytskich hipotez jest dosyc niskie gdyz odchylenie standardowe jest relatywnie wysokie.

In [17]:
data = auto.groupby(["foreign"]).agg({"price": list})
{a: stats.ttest_ind(data.loc["Domestic", "price"], data.loc["Foreign", "price"], alternative = a) for a in ['two-sided', 'less', 'greater']}

{'two-sided': Ttest_indResult(statistic=-0.4138898832983144, pvalue=0.6801850882546103),
 'less': Ttest_indResult(statistic=-0.4138898832983144, pvalue=0.34009254412730516),
 'greater': Ttest_indResult(statistic=-0.4138898832983144, pvalue=0.659907455872695)}

In [ ]:
%%stata
ttest price, by(foreign)

test t dla srednich bez zalozenia o rownej wariancji

In [ ]:
stats.ttest_ind(data.loc["Domestic", "price"], data.loc["Foreign", "price"], equal_var=False)

In [ ]:
%stata ttest price, by(foreign) unequal

Histogram zmiennej price

In [ ]:
# matplotlib has to be installed
auto.price.hist(bins = 10)

In [ ]:
%stata histogram price, bin(10)

Argument "by" moze zostac uzyty na przestrzeni wielu funkcji

In [ ]:
auto.hist(column = "price", bins = 10, by = "foreign")

In [ ]:
%stata histogram price, bin(10) by(foreign)

/* Czy rozklad zmiennej jest zblizony do rozkladu normalnego, jesli nie, to jaki ma rozklad? */

In [ ]:
auto["lprice"] = np.log(auto.price)

In [ ]:
%stata gen lprice = ln(price)

/* Czy po transformacji zmienna ma rozklad zblizony do normalnego? */

In [ ]:
stats.shapiro(auto.lprice), stats.shapiro(auto.price)

In [ ]:
%stata swilk lprice price

Przeprowadzmy test dla przeksztalconej zmiennej

In [ ]:
%%stata
ttest lprice, by(foreign)
ttest lprice, by(foreign) unequal

In [ ]:
data = auto.groupby(["foreign"]).agg({"lprice": list})
stats.ttest_ind(data.loc["Domestic", "lprice"], data.loc["Foreign", "lprice"]),\
stats.ttest_ind(data.loc["Domestic", "lprice"], data.loc["Foreign", "lprice"], equal_var=False)

Test dla proporcji - Dla jednej proby

Prosze zwrocic uwage ze na dole wskazane sa Hipotezy alternatywne a nie zerowe.

In [ ]:
# for level 1 - Foreign
%stata prtest foreign==.4
#  Stata liczy p-value z rozkładu asymptotycznego (normalnego)

In [ ]:
stats.binomtest(auto.foreign.value_counts()["Foreign"], auto.shape[0], .4)

In [ ]:
{a: proportions_ztest(auto.foreign.value_counts()["Foreign"].sum(), auto.shape[0], .4, alternative = a) for a in ["smaller", "two-sided", "larger"]}

Sprawdzamy, czy udzial samochodow o cenie pow. 10.000$ jest taki sam jak zagranicznych

In [ ]:
%%stata
/* Tworzymy zmienna 0-1 o wartosci 1 dla ceny pow. 10.000$ */
gen price_10 = (price>10000)
prtest price_10==foreign

In [ ]:
auto["price_10"] = auto.price > 10000
proportions_ztest([auto["price_10"].sum(), auto.foreign.value_counts()["Foreign"]], [auto.shape[0], auto.shape[0]])

Testy dla wariancji - Test F

In [ ]:
%stata sdtest price, by(foreign)

In [ ]:
data = auto.groupby(["foreign"]).agg({"price": list})
f_test(data.loc["Domestic", "price"], data.loc["Foreign", "price"])

Test Levene

In [ ]:
%%stata
/* W0 - wartosc statystyki Levene */
/* W50 - wartosc statystyki dla testu, w ktorym zamiast sredniej wykorzystano mediane */
/* W10 - wartosc statystyki dla testu, w ktorym zamiast sredniej wykorzystano ucieta srednia  */
robvar price, by(foreign)

In [ ]:
stats.f_oneway(data.loc["Domestic", "price"], data.loc["Foreign", "price"])

Testy dla rozkladow - Test Kurskalla - Wallisa

In [ ]:
%stata kwallis price, by(foreign)

In [ ]:
stats.kruskal(data.loc["Domestic", "price"], data.loc["Foreign", "price"])

Test Kolmogorowa - Smirnowa

In [ ]:
%stata ksmirnov price, by(foreign)

In [ ]:
# - `two-sided`: The null hypothesis is that the two distributions are
#   identical, F(x)=G(x) for all x; the alternative is that they are not
#   identical.
# - `less`: The null hypothesis is that F(x) >= G(x) for all x; the
#   alternative is that F(x) < G(x) for at least one x.
# - `greater`: The null hypothesis is that F(x) <= G(x) for all x; the
#   alternative is that F(x) > G(x) for at least one x.
{ a:stats.ks_2samp(data.loc["Domestic", "price"], data.loc["Foreign", "price"], alternative = a) for a in ["less", "two-sided", "greater"]}
# roznice na pvalue

### Cwiczenie
Na postawie danych z 4 kwartalu 2009 roku bedziemy porownywac:  
1. place kobiet i mezczyzn w Polsce
2. wyksztalcenie kobiet i mezczyzn w Polsce
Prosze pobrac dane ze strony www.ekonometria.wne.uw.edu.pl i zaladowac do Staty  

Korzystajac z odpowiednich testow, prosze sprawdzic:
- Czy srednia placa kobiet jest rowna sredniej placy mezczyzn?
- Czy zroznicowanie plac kobiet jest rowne zroznicowaniu plac mezczyzn?
- Czy rozklad plac kobiet jest rowny rozkladowi plac mezczyzn?
- Kto jest lepiej wyksztalcony: kobiety czy mezczyzni?
- Czy zroznicowanie poziomu wyksztalcenia jest wieksze wsrod kobiet czy wsrod mezczyzn?
- Czy rozklad wyksztalcenia kobiet jest rowny rozkladowi wyksztalcenia mezczyzn?


In [ ]:
%stata clear

In [ ]:
# %stata use ...
# pd.read

Kilka ulatwien

In [ ]:
# %%stata
# /* Zmiana nazwy zmiennej */
# rename pl plec
# /* Utworzenie etykiety */
# label define plec_lab 1 "Mezczyzna" 2 "Kobieta"
# /* Nadanie etykiety wartosciom zmiennej */
# label values plec plec_lab
# /* utworzenie zmiennej placa, w bazie wartosc "99999" oznacza odmowe odpowiedzi */
# gen placa = P37 if (P37!=99999)

## Cwiczenie
Bedziemy pracowac na danych z badania Diagnoza Spoleczna z 2011 roku  
Prosze pobrac dane ze strony www.ekonometria.wne.uw.edu.pl i zaladowac zbior osob do Staty  
Prosze zapoznac sie z zawartoscia zbioru i odpowiedziec na pytania, przeprowadzajac odpowiednie testy
- Czy kobiety przecietnie wiecej waza od mezczyzn?
- Czy kobiety sa przecietnie wyzsze od mezczyzn? 
- Czy kobiety sa lepiej wyksztalcone od mezczyzn? 
- Czy wsrod osob o wyzszym wyksztalceniu kobiety przecietnie wiecej waza od mezczyzn? 
- Czy wsrod osob o wyzszym wyksztalceniu kobiety sa przecietnie wyzsze od mezczyzn? 
- Czy wsrod osob o wyzszym wyksztalceniu kobiety wiecej zarabiaja od mezczyzn? 
